In [16]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats # statistical tests


################### Linear Models#######################
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge, BayesianRidge 

################### Non-Linear Models#######################
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingRegressor 

from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error


In [17]:
#import datasets

X = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_log = pd.read_csv('y_log.csv')
y = pd.read_csv('y.csv')


X.head()


,Unnamed: 0,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,Fireplaces,GarageCars,...,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed
0,0,2003,2003,856.0,856,1710,2,8,0,2.0,...,0,0,0,1,0,1,0,0,0,0
1,1,1976,1976,1262.0,1262,1262,2,6,1,2.0,...,0,0,0,1,0,1,0,0,0,0
2,2,2001,2002,920.0,920,1786,2,6,1,2.0,...,0,0,0,1,0,1,0,0,0,0
3,3,1915,1970,756.0,961,1717,1,7,1,3.0,...,0,0,0,1,0,1,0,0,0,0
4,4,2000,2000,1145.0,1145,2198,2,9,1,3.0,...,0,0,0,1,0,1,0,0,0,0


In [19]:
X_test.head()

,Id,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,Fireplaces,GarageCars,...,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed
0,1461.0,1961,1961,882.0,896,896,1,5,0,1.0,...,0,0,0,1,0,1,0,0,0,0
1,1462.0,1958,1958,1329.0,1329,1329,1,6,0,1.0,...,0,0,0,1,0,0,0,1,0,0
2,1463.0,1997,1998,928.0,928,1629,2,6,1,2.0,...,0,0,0,1,0,1,0,0,0,0
3,1464.0,1998,1998,926.0,926,1604,2,7,1,2.0,...,0,0,0,1,0,1,0,0,0,0
4,1465.0,1992,1992,1280.0,1280,1280,2,5,0,2.0,...,0,0,0,1,0,1,0,0,0,0


In [20]:
y_log.head()

,Unnamed: 0,Log_SalePrice
0,0,12.247699
1,1,12.109016
2,2,12.317171
3,3,11.849405
4,4,12.429220


In [21]:
y.head()

,Unnamed: 0,SalePrice
0,0,208500.0
1,1,181500.0
2,2,223500.0
3,3,140000.0
4,4,250000.0


In [22]:
def cv_rmse(model):
    scores_rmse = np.sqrt(-cross_val_score(model, X, y_log, scoring="neg_mean_squared_error", cv=kfold, n_jobs=4))
    mean_rmse = scores_rmse.mean()
    std_rmse = scores_rmse.std()
    return mean_rmse, std_rmse

In [23]:
cv_score = []

kfold = KFold(n_splits=10, random_state=3, shuffle=True)

baseline_models = [('Ridge', Ridge()), ('Lasso', Lasso()), ('ElasticNet', ElasticNet()), 
                   ('BayesianRidge', BayesianRidge()), ('SVR', SVR()), ('KernelRidge', KernelRidge()),
                    ('RandomForestRegressor', RandomForestRegressor()), ('GradientBoostingRegressor', GradientBoostingRegressor()), 
                    ]

for idx, (name, model) in enumerate(baseline_models):
    if idx < 6:
        pipe = make_pipeline(RobustScaler(), model)
        cv_score.append((name, *cv_rmse(pipe)))
    else: 
        cv_score.append((name, *cv_rmse(model)))   

cv_scores = pd.DataFrame(data=cv_score, columns=['model', 'mean_rmse', 'std_rmse']).sort_values('mean_rmse').set_index('model')
cv_scores

/Users/felipelastra/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/felipelastra/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/felipelastra/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/felipelastra/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_bayes.py",

,mean_rmse,std_rmse
model,,
RandomForestRegressor,0.716085,0.047237
Ridge,0.808285,0.024885
Lasso,1.258059,0.034374
KernelRidge,6.570480,1.672822
ElasticNet,180.538008,4.714509
BayesianRidge,NaN,NaN
SVR,NaN,NaN
GradientBoostingRegressor,NaN,NaN
